In [118]:
seed=40
embedding_dim = 100
vocab_size = 12112
lower = False
FN0 = 'train_data'
FN = 'vocabulary-embedding'

In [119]:
import _pickle as pickle
with open('data/all_input.pkl', 'rb') as contentFile:
    (X, Y) = pickle.load(contentFile)

In [120]:
print(X[1], Y[1])

['Amazing', '!', 'I', 'was', 'swept', 'away', 'when', 'I', 'walked', 'into', 'the', 'hotel', 'it', 'was', 'gorgeous', '.', 'The', 'staff', 'was', 'very', 'nice', 'and', 'helpful', 'the', 'hotel', 'was', 'very', 'clean', ',', 'and', 'the', 'food', 'was', 'delicious', '.', 'We', 'stayed', 'two', 'nights', 'and', 'the', 'first', 'night', 'we', 'went', 'to', 'the', 'Lockwood', 'restaurant', 'and', 'we', 'were', 'blown', 'away', 'by', 'the', 'food', '.', 'It', 'has', 'to', 'be', 'the', 'best', 'food', 'in', 'Chicago', 'the', 'atmosphere', 'was', 'very', 'classy', 'and', 'warm', 'and', 'the', 'wait', 'staff', 'was', 'very', 'precise', '.', 'The', 'decor', 'in', 'our', 'rooms', 'did', 'not', 'look', 'like', 'any', 'hotel', 'I', 'had', 'been', 'at', 'they', 'were', 'very', 'elegant', 'and', 'inviting', '.', 'I', 'can', 'not', 'wait', 'to', 'go', 'back', 'it', 'is', 'a', 'true', 'escape', 'from', 'daily', 'life', '.'] False


In [121]:
if lower:
    content = [h.lower() for h in X]

In [122]:
print(len(X))

1600


In [123]:
from collections import Counter
from itertools import chain
def get_vocab(lst):
    vocabcount = Counter(w for txt in lst for w in txt)
    vocab = list(map(lambda x: x[0], sorted(vocabcount.items(), key=lambda x: -x[1])))
    return vocab, vocabcount

In [124]:
vocab, vocabcount = get_vocab(X)

In [125]:
print (vocab[:50])
print ('...',len(vocab))

['.', 'the', ',', 'and', 'to', 'I', 'a', 'was', 'in', 'of', 'The', 'for', 'hotel', 'room', 'at', 'it', 'is', 'we', 'that', 'my', 'were', 'with', 'had', 'not', '!', 'on', 'Chicago', 'this', 'but', 'stay', 'very', 'our', 'have', 'We', "n't", 'you', 'they', 'be', 'there', 'would', 'from', 'as', 'are', 'all', 'me', 'staff', 'great', 'so', 'service', 'did']
... 12110


In [126]:
i = 0
for key in vocab:
    print(key, vocabcount[key])
    if(i > 10):
        break
    i += 1

. 13678
the 12806
, 8715
and 7802
to 6702
I 6357
a 6331
was 5986
in 3784
of 3287
The 3145
for 2786


In [127]:

"""with open('data/vocabList.pkl', 'w+b') as output_file:
    pickle.dump(vocab, output_file)
with open('data/vocab_counter.pkl', 'w+b') as output_file:
    pickle.dump(vocabcount, output_file)
print('finished')"""

"with open('data/vocabList.pkl', 'w+b') as output_file:\n    pickle.dump(vocab, output_file)\nwith open('data/vocab_counter.pkl', 'w+b') as output_file:\n    pickle.dump(vocabcount, output_file)\nprint('finished')"

In [128]:
print('here')

here


In [129]:
empty = 0 # RNN mask of no data
eos = 1  # end of sentence
start_idx = eos+1 # first real word

In [130]:
def get_idx(vocab, vocabcount):
    word2idx = dict((word, idx+start_idx) for idx,word in enumerate(vocab))
    word2idx['<empty>'] = empty
    word2idx['<eos>'] = eos
    
    idx2word = dict((idx,word) for word,idx in word2idx.items())

    return word2idx, idx2word

In [131]:
word2idx, idx2word = get_idx(vocab, vocabcount)

In [132]:
"""with open('data/word2idx.pkl', 'w+b') as output_file:
    pickle.dump(word2idx, output_file)
with open('data/idx2word.pkl', 'w+b') as output_file:
    pickle.dump(idx2word, output_file)
print('finished')"""

"with open('data/word2idx.pkl', 'w+b') as output_file:\n    pickle.dump(word2idx, output_file)\nwith open('data/idx2word.pkl', 'w+b') as output_file:\n    pickle.dump(idx2word, output_file)\nprint('finished')"

In [133]:
import os
cwd = os.getcwd()

In [134]:
fname = '/data/glove.6b/glove.6B.%dd.txt'%embedding_dim
fname = cwd + fname

In [135]:
glove_n_symbols = !wc -l {fname}

In [136]:
glove_n_symbols = int(glove_n_symbols[0].split()[0])
glove_n_symbols

400000

In [137]:
import numpy as np
glove_index_dict = {}
glove_embedding_weights = np.empty((glove_n_symbols, embedding_dim))
globale_scale=.1
with open(fname, 'r') as fp:
    i = 0
    for l in fp:
        l = l.strip().split()
        w = l[0]
        glove_index_dict[w] = i
        glove_embedding_weights[i,:] = np.array(list(map(float,l[1:])))
        i += 1
glove_embedding_weights *= globale_scale

In [138]:
glove_embedding_weights.std()

0.040815727600190289

In [139]:
for w,i in glove_index_dict.items():
    w = w.lower()
    if w not in glove_index_dict:
        glove_index_dict[w] = i

In [140]:
# generate random embedding with same scale as glove
np.random.seed(seed)
shape = (vocab_size, embedding_dim)
scale = glove_embedding_weights.std()*np.sqrt(12)/2 # uniform and not normal
embedding = np.random.uniform(low=-scale, high=scale, size=shape)
print ('random-embedding/glove scale', scale, 'std', embedding.std())

# copy from glove weights of words that appear in our short vocabulary (idx2word)
c = 0
for i in range(vocab_size):
    w = idx2word[i]
    g = glove_index_dict.get(w, glove_index_dict.get(w.lower()))
    if g is None and w.startswith('#'): # glove has no hastags (I think...)
        w = w[1:]
        g = glove_index_dict.get(w, glove_index_dict.get(w.lower()))
    if g is not None:
        embedding[i,:] = glove_embedding_weights[g,:]
        c+=1
print ('number of tokens, in small vocab, found in glove and copied to embedding', c,c/float(vocab_size))

random-embedding/glove scale 0.0706949139514 std 0.0408301773961
number of tokens, in small vocab, found in glove and copied to embedding 10752 0.8877146631439894


In [141]:
embedding_size = embedding.shape

In [142]:
glove_thr = 0.5

In [143]:
word2glove = {}
for w in word2idx:
    if w in glove_index_dict:
        g = w
    elif w.lower() in glove_index_dict:
        g = w.lower()
    elif w.startswith('#') and w[1:] in glove_index_dict:
        g = w[1:]
    elif w.startswith('#') and w[1:].lower() in glove_index_dict:
        g = w[1:].lower()
    else:
        continue
    word2glove[w] = g

In [144]:
normed_embedding = embedding/np.array([np.sqrt(np.dot(gweight,gweight)) for gweight in embedding])[:,None]

nb_unknown_words = 100

glove_match = []
for w,idx in word2idx.items():
    if idx >= vocab_size-nb_unknown_words and w.isalpha() and w in word2glove:
        gidx = glove_index_dict[word2glove[w]]
        gweight = glove_embedding_weights[gidx,:].copy()
        # find row in embedding that has the highest cos score with gweight
        gweight /= np.sqrt(np.dot(gweight,gweight))
        score = np.dot(normed_embedding[:vocab_size-nb_unknown_words], gweight)
        while True:
            embedding_idx = score.argmax()
            s = score[embedding_idx]
            if s < glove_thr:
                break
            if idx2word[embedding_idx] in word2glove :
                glove_match.append((w, embedding_idx, s)) 
                break
            score[embedding_idx] = -1
glove_match.sort(key = lambda x: -x[2])
print ('# of glove substitutes found', len(glove_match))

# of glove substitutes found 71


In [145]:
for orig, sub, score in glove_match[-20:-10]:
    print (score, orig,'=>', idx2word[sub])

0.645501756934 mugs => stickers
0.636291306623 Cheryl => Donna
0.633960283289 bunk => bed
0.619100872829 PDA => iPod
0.615784546512 serviceable => passable
0.614677928838 hides => hide
0.612279810942 spit => chewed
0.607427658303 submitting => requesting
0.606467005816 swung => pushed
0.593433076334 claw => bite


In [147]:
glove_idx2idx = dict((word2idx[w],embedding_idx) for  w, embedding_idx, _ in glove_match)

In [148]:
X = [[word2idx[token] for token in content] for content in X]
len(X)

1600

In [149]:
import _pickle as pickle
with open('data/%s.pkl'%FN,'wb') as fp:
    pickle.dump((embedding, idx2word, word2idx, glove_idx2idx),fp,-1)

In [150]:
import _pickle as pickle
with open('data/%s.data.pkl'%FN,'wb') as fp:
    pickle.dump((X,Y),fp,-1)

In [151]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=320, random_state=seed)

In [152]:
len(X_train), len(Y_train), len(X_test), len(Y_test)

(1280, 1280, 320, 320)

In [153]:
print(X_train[0])

[90, 18, 56, 1106, 14, 4, 5, 3, 2748, 1595, 2, 863, 44, 159, 4, 5, 8, 285, 2772, 2, 1674, 10, 21, 15, 9, 10, 9836, 245, 11, 663, 2, 1498, 9, 117, 4, 221, 4, 5, 3, 15, 9, 452, 4, 64, 47, 9, 825, 4, 30, 25, 114, 1786, 10, 3, 310, 2, 583, 3, 300, 9, 3726, 2, 1523, 9, 10382, 541, 13, 3, 226, 910, 11, 186, 10, 436, 2, 1437, 4, 8, 14, 11096, 2639, 49, 134, 10, 248, 6079, 11170, 261, 1233, 137, 2, 12, 14, 114, 1114, 672, 6, 39, 674, 2]


In [154]:
import keras
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
maxlen = 300
trainX = pad_sequences(X_train, maxlen=maxlen, value=0.)
testX = pad_sequences(X_test, maxlen=maxlen, value=0.)

trainY = keras.utils.to_categorical(Y_train, num_classes=2)
testY = keras.utils.to_categorical(Y_test, num_classes=2)


In [79]:
#embedding = tf.cast(embedding, tf.float32)

In [155]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector, Flatten, Lambda
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D
from keras.regularizers import l2
from keras.layers import SpatialDropout1D
import random
import keras.backend as K
from keras import layers

In [165]:
random.seed(seed)
np.random.seed(seed)
p_W, p_U, p_dense, p_emb, weight_decay = 0.2, 0.2, 0, 0, 0.0
optimizer = 'rmsprop'
LR = 1e-4
batch_size=64
rnn_size = 128
rnn_layers = 1

In [166]:
regularizer = l2(weight_decay) if weight_decay else None

In [167]:
print(regularizer)

None


In [168]:
model = None
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,#mask_zero=False,
                    input_length=maxlen,
                    embeddings_regularizer=regularizer,
                    name='embedding_1'))
model.add(Dropout(p_emb))
model.add(Conv1D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=False, # batch_norm=batch_norm,
                recurrent_regularizer=regularizer, kernel_regularizer=regularizer,
                bias_regularizer=regularizer, recurrent_dropout=p_W, dropout=p_U,
                name='lstm_%d'%(i+1)
                  )
    model.add(lstm)

model.add(Dense(2, activation='softmax'))

In [169]:
def str_shape(x):
    return 'x'.join(map(str,x.shape))
    
def inspect_model(model):
    for i,l in enumerate(model.layers):
        print (i, 'cls=%s name=%s'%(type(l).__name__, l.name))
        weights = l.get_weights()
        for weight in weights:
            print (str_shape(weight)),
        print

In [170]:
from keras.optimizers import Adam, RMSprop # usually I prefer Adam but article used rmsprop
# opt = Adam(lr=LR)  # keep calm and reduce learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [171]:

K.set_value(model.optimizer.lr,np.float32(LR))

In [172]:
inspect_model(model)

0 cls=Embedding name=embedding_1
12112x100
1 cls=Dropout name=dropout_9
2 cls=Conv1D name=conv1d_5
2x100x64
64
3 cls=MaxPooling1D name=max_pooling1d_4
4 cls=LSTM name=lstm_1
64x512
128x512
512
5 cls=Dense name=dense_8
128x2
2


In [174]:
model.fit(trainX, trainY, validation_data = (testX, testY), epochs = 50, verbose=1)

Train on 1280 samples, validate on 320 samples
Epoch 1/50
1280/1280 [==============================] - 16s - loss: 0.2309 - acc: 0.9375 - val_loss: 0.5567 - val_acc: 0.8344

In [175]:
model.evaluate(testX, testY)

320/320 [==============================] - 0s     


[0.48902042210102081, 0.84375]

In [ ]:
model.save('lstm_cnn')

In [38]:
"""net = tflearn.input_data([None, maxlen])
net = tflearn.embedding(net, input_dim=vocab_size, output_dim=128)
net = tflearn.lstm(net, 128, dropout = 0.7)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy')"""

In [39]:
"""len(trainX), len(trainY), len(testX), len(testY)"""

(1440, 1440, 160, 160)

In [41]:
"""model = tflearn.DNN(net, tensorboard_verbose=0, best_checkpoint_path = os.getcwd() + '/checkpoint/', 
                    best_val_accuracy=0.8)"""

In [42]:
"""model.load(os.getcwd() + '/checkpoint/5750')"""

"model.load(os.getcwd() + '/checkpoint/5750')"

In [43]:
"""model.evaluate(testX, testY)[0]"""

0.55000000000000004

In [44]:
"""numberOfRuns = 1
while(model.evaluate(testX, testY)[0]<0.89 and model.evaluate(trainX, trainY)[0]<0.99 and numberOfRuns <= 20):
    model.fit(trainX, trainY, validation_set=(testX, testY), show_metric=True, n_epoch = 50)
    print(model.evaluate(testX, testY))
    numberOfRuns+=1
    """
    

Training Step: 1609  | total loss: 0.65045 | time: 11.953s
| Adam | epoch: 070 | loss: 0.65045 - acc: 0.5617 -- iter: 1408/1440
Training Step: 1610  | total loss: 0.64975 | time: 13.541s
| Adam | epoch: 070 | loss: 0.64975 - acc: 0.5711 | val_loss: 0.76642 - val_acc: 0.4688 -- iter: 1440/1440
--


KeyboardInterrupt: 

In [ ]:
#import os
#model.save(os.getcwd())

In [ ]:
"""print('number of iterations = ' + str(numberOfRuns*50))
print('validation accuracy = ' + str(model.evaluate(testX, testY)[0]*100) + '%')"""
